# Загрузка Pandas и очистка данных

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('main_task.csv')

In [3]:
cities = {c: i for i, c in enumerate(df.City.value_counts().index.tolist())}
df['CityID'] = df.City.apply(lambda x: cities[x])

df['Number of Reviews'] = df['Number of Reviews'].fillna(df['Number of Reviews'].median())

In [4]:
df['Cuisine Style'] = df['Cuisine Style'].fillna("[]")
import json
def extract_cs(cs):
    csl = cs[1:-1].replace('\'', '').split(',')
    return len(list(map(lambda x: x.strip(), csl)))

df['cousines_count'] = df['Cuisine Style'].apply(extract_cs)

In [5]:
all_cuisines = set()
df['Cuisine Style'] = df['Cuisine Style'].fillna("[]")
for row in df['Cuisine Style'].values:
    cuisines = list(map(lambda x: x.strip(), row[1:-1].replace('\'', '').split(',')))
    list(map(all_cuisines.add, cuisines))
    
    
def has_cuisine(cuisines, target):
    return 1 if target in cuisines else 0
    
    
for cuisine in all_cuisines:
    if not cuisine:
        continue
    df[cuisine] = df['Cuisine Style'].apply(lambda x: has_cuisine(x, cuisine))

In [6]:
capitals = pd.read_csv('capitals.csv')
capitals.head()
capitals_set = capitals.capital.str.lower().unique()

def is_capital(city):
    return city.lower() in capitals_set

df['capital'] = df.City.apply(lambda x: 1 if is_capital(x) else 0)

In [7]:
import json
df.Reviews = df.Reviews.fillna('[[], []]')

def extract_days_count(x):
    data = x.split('], [')[-1][1:-3].split('\', \'')
    if not all(data):
        return 0
    try:
        d1 = pd.to_datetime(data[0], format='%m/%d/%Y')
        d2 = pd.to_datetime(data[1], format='%m/%d/%Y')
        result = (d1 - d2).days
    except IndexError:
        result = 0
    return result

In [14]:
# def map_price(x):
#     if x == '$$ - $$$':
#         return 1
#     if x == '$': 
#         return 2
#     if x == '$$$$':
#         return 3
#     return 0

# df['Price Range'] = df['Price Range'].fillna('$$ - $$$')
# df['price_map'] = df['Price Range'].apply(map_price)

In [8]:
df = df.drop(['City', 'Reviews', 'URL_TA', 'ID_TA', 'Price Range', 'Cuisine Style'], axis=1)

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [9]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Restaurant_id', 'Rating'], axis = 1)
y = df['Rating']

In [10]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [11]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [12]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [13]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [14]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.2114195
